In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import re


In [ ]:
model = pd.read_csv('https://raw.githubusercontent.com/Adelaaas/Data_science_advanced_2022/main/%D0%9D%D0%A2%D0%9E%20%D0%B8%D0%BD%D0%B6%20%D1%82%D1%83%D1%80/train.csv')
check = pd.read_csv('https://raw.githubusercontent.com/Adelaaas/Data_science_advanced_2022/main/%D0%9D%D0%A2%D0%9E%20%D0%B8%D0%BD%D0%B6%20%D1%82%D1%83%D1%80/predict.csv')

In [ ]:
model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31641 entries, 0 to 31640
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Vmag         31641 non-null  float64
 1   Plx          31641 non-null  float64
 2   e_Plx        31641 non-null  float64
 3   B-V          31641 non-null  float64
 4   SpType       31641 non-null  object 
 5   Amag         31641 non-null  float64
 6   TargetClass  31641 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.7+ MB


In [ ]:
model.head(10)

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,6.901420,24.569658,0.862829,0.737416,G5V,20.449126,1
1,4.334196,2.198947,0.593309,1.153210,G2Ib,13.043324,0
2,7.873019,11.750465,0.616568,0.589166,F5V,17.736492,1
3,8.110014,2.143815,0.848508,1.322107,K2III,15.592469,0
4,8.025193,4.165622,0.899296,1.141646,K5III,16.379680,0
5,2.899130,11.110086,0.701719,1.095357,M0III,14.370388,0
6,7.529711,9.160176,1.131393,0.926519,G8IV,15.938899,1
7,6.083824,3.928857,0.835288,1.761544,M2.5IIIa,14.063334,0
8,3.000914,5.713252,0.757593,0.113611,A0V,14.350210,1
9,8.706528,1.816360,1.409039,0.250017,A0V,14.763104,1


In [ ]:
check.head(10)

,Vmag,Plx,e_Plx,B-V,SpType,Amag
0,4.039519,13.596504,0.845101,-0.097823,F5V+...,15.774122
1,6.655719,4.010332,1.050017,1.386670,K5III,15.344976
2,6.405060,0.348855,0.961598,0.951503,B0IVpe,12.236103
3,9.113068,2.954461,1.251422,1.002819,K1III,15.560838
4,6.107686,3.144180,0.475795,0.493345,G8II-IIIvar,12.771981
5,7.337771,12.254347,0.651041,0.330493,F4V,17.315567
6,7.316560,0.662464,0.720464,-0.056743,B4V:ne,12.845317
7,7.379830,4.494505,0.594863,1.322639,K2III,15.528721
8,9.284617,7.392761,1.116969,0.629743,G3V,19.626463
9,8.646860,4.137044,1.239575,0.371263,F6V:,17.284021


In [ ]:
model['Class'] = 0
model['illum'] = 0


for index, row in model.iterrows():
    model.loc[index, 'Class'] = re.findall(r'([A-Z])', model.loc[index, 'SpType'])[0]
    try:
        b = re.findall(r'([0-9])', model.loc[index, 'SpType'])[0]
        a = re.findall(r'.*[0-9]([IV]*)', model.loc[index, 'SpType'])[-1]
        if a not in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']:
            model.loc[index, 'illum'] = 'IV'
        else:
            model.loc[index, 'illum'] = a
        model.loc[index, 'class_class'] = b
    except IndexError:
        model.loc[index, 'illum'] = 'IV'
        model.loc[index, 'class_class'] = 4
sptype = model.drop("SpType", axis=1, inplace=True)

model

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,Class,illum,class_class
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,G,V,5
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,G,I,2
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,F,V,5
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,K,III,2
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,K,III,5
...,...,...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,12.542120,0,A,II,0
31637,8.078011,3.002784,0.689891,0.971966,15.379710,0,G,III,8
31638,7.065964,7.820498,0.885794,0.435381,16.737442,1,F,IV,3
31639,8.685452,2.643033,1.064647,0.988600,14.859882,0,K,III,0


In [ ]:
le1 = LabelEncoder()
le2 = LabelEncoder()
label1 = le1.fit_transform(model['Class'])
label2 = le2.fit_transform(model['illum'])
model.drop("Class", axis=1, inplace=True)
model.drop("illum", axis=1, inplace=True)
model['Class'] = label1
model['illum'] = label2

model

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,class_class,Class,illum
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,5,4,4
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,2,4,0
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,5,3,4
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,2,5,2
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,5,5,2
...,...,...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,12.542120,0,0,0,1
31637,8.078011,3.002784,0.689891,0.971966,15.379710,0,8,4,2
31638,7.065964,7.820498,0.885794,0.435381,16.737442,1,3,3,3
31639,8.685452,2.643033,1.064647,0.988600,14.859882,0,0,5,2


In [ ]:
y = model['TargetClass']
X = model.drop(columns = ['TargetClass'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

In [ ]:
%%time
rfc = RandomForestClassifier(random_state = 42, n_jobs = -1)
rfc.fit(X_train, y_train)

CPU times: user 4.55 s, sys: 34.6 ms, total: 4.58 s
Wall time: 2.48 s


RandomForestClassifier(n_jobs=-1, random_state=42)

In [ ]:
pred = rfc.predict(X_test)

In [ ]:
roc_auc_score(y_test, pred)

0.9593466452071661

In [ ]:
check['Class'] = 0
check['illum'] = 0


for index, row in check.iterrows():
    check.loc[index, 'Class'] = re.findall(r'([A-Z])', check.loc[index, 'SpType'])[0]
    try:
        b = re.findall(r'([0-9])', check.loc[index, 'SpType'])[0]
        a = re.findall(r'.*[0-9]([IV]*)', check.loc[index, 'SpType'])[-1]
        if a not in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']:
            check.loc[index, 'illum'] = 'IV'
        else:
            check.loc[index, 'illum'] = a
        check.loc[index, 'class_class'] = b
    except IndexError:
        check.loc[index, 'illum'] = 'IV'
        check.loc[index, 'class_class'] = 4
sptype = check.drop("SpType", axis=1, inplace=True)

check

,Vmag,Plx,e_Plx,B-V,Amag,Class,illum,class_class
0,4.039519,13.596504,0.845101,-0.097823,15.774122,F,V,5
1,6.655719,4.010332,1.050017,1.386670,15.344976,K,III,5
2,6.405060,0.348855,0.961598,0.951503,12.236103,B,IV,0
3,9.113068,2.954461,1.251422,1.002819,15.560838,K,III,1
4,6.107686,3.144180,0.475795,0.493345,12.771981,G,II,8
...,...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,16.438014,M,III,3
7907,6.255358,3.034436,0.449386,0.966053,13.898403,K,III,0
7908,8.643598,10.932145,0.947897,0.504383,18.980729,G,V,0
7909,9.400810,6.279361,1.332124,0.351073,16.872491,F,V,2


In [ ]:
le1 = LabelEncoder()
le2 = LabelEncoder()
label1 = le1.fit_transform(check['Class'])
label2 = le2.fit_transform(check['illum'])
check.drop("Class", axis=1, inplace=True)
check.drop("illum", axis=1, inplace=True)
check['Class'] = label1
check['illum'] = label2

check

,Vmag,Plx,e_Plx,B-V,Amag,class_class,Class,illum
0,4.039519,13.596504,0.845101,-0.097823,15.774122,5,3,4
1,6.655719,4.010332,1.050017,1.386670,15.344976,5,5,2
2,6.405060,0.348855,0.961598,0.951503,12.236103,0,1,3
3,9.113068,2.954461,1.251422,1.002819,15.560838,1,5,2
4,6.107686,3.144180,0.475795,0.493345,12.771981,8,4,1
...,...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,16.438014,3,6,2
7907,6.255358,3.034436,0.449386,0.966053,13.898403,0,5,2
7908,8.643598,10.932145,0.947897,0.504383,18.980729,0,4,4
7909,9.400810,6.279361,1.332124,0.351073,16.872491,2,3,4


In [ ]:
pred = rfc.predict(check)

In [ ]:
pred = pd.Series(pred)

In [ ]:
pred.head(10)

0    1
1    0
2    1
3    0
4    0
5    1
6    1
7    0
8    1
9    1
dtype: int64

In [ ]:
pred.to_csv('Result.csv')
pred_file = pd.read_csv('/content/Result.csv')
pred_file.head(10)

,Unnamed: 0,0
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
5,5,1
6,6,1
7,7,0
8,8,1
9,9,1


In [ ]:
pred_file.rename(columns={'0': 'TargetClass','Unnamed: 0':'Number'}, inplace=True)

In [ ]:
pred_file.head(10)

,Number,TargetClass
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
5,5,1
6,6,1
7,7,0
8,8,1
9,9,1


Удалим ненужные столбцы.

In [ ]:
check=check.join(pred_file)

In [ ]:
check.head()

,Vmag,Plx,e_Plx,B-V,Amag,class_class,Class,illum,Number,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,15.774122,5,3,4,0,1
1,6.655719,4.010332,1.050017,1.386670,15.344976,5,5,2,1,0
2,6.405060,0.348855,0.961598,0.951503,12.236103,0,1,3,2,1
3,9.113068,2.954461,1.251422,1.002819,15.560838,1,5,2,3,0
4,6.107686,3.144180,0.475795,0.493345,12.771981,8,4,1,4,0


In [ ]:
pred_final = check.drop(['Number'], axis='columns')
pred_final = pred_final.reindex(columns=['Vmag', 'Plx', 'e_Plx', 'B-V', 'SpType', 'Amag', 'TargetClass'])


In [ ]:
pred_final.head(10)

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,NaN,15.774122,1
1,6.655719,4.010332,1.050017,1.386670,NaN,15.344976,0
2,6.405060,0.348855,0.961598,0.951503,NaN,12.236103,1
3,9.113068,2.954461,1.251422,1.002819,NaN,15.560838,0
4,6.107686,3.144180,0.475795,0.493345,NaN,12.771981,0
5,7.337771,12.254347,0.651041,0.330493,NaN,17.315567,1
6,7.316560,0.662464,0.720464,-0.056743,NaN,12.845317,1
7,7.379830,4.494505,0.594863,1.322639,NaN,15.528721,0
8,9.284617,7.392761,1.116969,0.629743,NaN,19.626463,1
9,8.646860,4.137044,1.239575,0.371263,NaN,17.284021,1


In [ ]:
pred_final.to_csv('Result_final.csv',index=False)